In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

## In Defense of VaR: 

People like to criticize value at risk, and there has been a big push from certain quarters to favor expected shortfall (aka ES, conditional VaR, or cVaR) over VaR. I think this push is misguided (VaR is not as bad as they make it out to be and ES has its own shortcomings). 
 
For this article I wanted to look at how the ratio of ES over VaR changes over time. I suspect most practitioners expect this ratio to increase in stress environments (ES is supposed to tell us a lot about the shape of tails; we associate stress environments with tail risk; therefore ES should "light up" more in a stress environment than VaR). I wonder if the opposite is not true.
 
To start, we could look at  realized VaR and ES for the S&P 500 in each calendar year from 1995 to present (you'd have at least three big market declines: tech bubble bursting in 1999, financial crisis in 2008, and current market). Depending on initial results, we could look at other indices (other countries, other asset classes) and/or hypothetical long/short portfolios.

### What is Value at Risk (VaR)

A measure called value at Risk (or VaR) was created to answer the question "how bad can things get realistically." It's now widely used and, in some cases, required by financial regulators to be computed and acted on.

VaR is simply a percentile of a probability distribution of a financial quantity of interest. The quantity is usually either:
- _P&L_ (profit&loss, i.e. change in wealth from the current wealth); or
- _Rate of return_ or just _return_ of wealth; that's ending wealth divided by beginning wealth, all minus one;
- _Log-return_, which is the logarithm of one plus the rate of return.

Note that all of these quantities can in general take on both positive and negative values. Value at Risk is concerned with how much can be lost, so the convention is to reverse the sign in a way we'll describe below, giving bigger positive numbers for bigger losses.

Let $X$ be the random variable giving the future value of whichever one of the quantities above is being analyzed, and let $F_t(x)$ be the cumulative distribution function (cdf) of $X$'s value at some time $t$ in the future. The Value at Risk (VaR) over time $t>0$ with probability $0\leq p\leq 1$ is:
$$VaR_t(p)=-\inf\{x\mid F_t(x)=Pr(X\le x)\ge 1-p\}$$

If $X$'s cdf $F_t$ is continuous, then the calculation simplifies to:
$$VaR_t(p)=-F_t^{-1}(1-p);\hspace{2em}VaR_t(p)=x \iff \int_{-\infty}^{-x}f_t(y)dy=1-p$$
where $f_t(y)$ is the probability density function of the distribution.

In words, we say that $VaR_t(p)$ is the **`t-year p Value at Risk`**. $t$ could also be denominated in other time units like days. The time argument $t$ might be left implicit if it is specified elsewhere.


### An example of VaR

For example, if a bank wants to estimate how bad things can get in its trading operations, the random variable of interest might be $\Delta w$, the change (P&L) in its trading capital $w$ by the end of tomorrow's trading day. The one-day $99\%$ VaR for its trading operations would be the (positive) loss amount that was expected to exceed $-\Delta w$, $99$ days out of $100$. So if the one-day $99\%$ VaR is $\$50,000,000$, then the bank expects to lose less than $\$50,000,000$ on all but one out of $100$ trading days.

### realized VaR and ES for the S&P 500 in each calendar year from 1995 to present

In [120]:
# SP500
# surprisingly though, popular free data feeds (AlphaVantage, quandl, FRED) only give historical data for the past 20 years.
# so I downloaded all historical data from yahoo finance for now
df_sp500 = pd.read_csv('SP500.csv', index_col=0, parse_dates=True, infer_datetime_format=True, header=0, usecols=['Date', 'Adj Close'])
# 1995/1/1 - today
start_date = pd.to_datetime('1995-1-1')
end_date = date.today()
df_sp500 = df_sp500.loc[start_date:end_date]
df_sp500

,Adj Close
Date,
1995-01-03,459.109985
1995-01-04,460.709991
1995-01-05,460.339996
1995-01-06,460.679993
1995-01-09,460.829987
...,...
2020-05-22,2955.449951
2020-05-26,2991.770020
2020-05-27,3036.129883


In [121]:
time_period = 252 # days
p = 5 # p := VaR(1-p)

In [122]:
# compute profit/loss
df_PL = (df_sp500-df_sp500.shift(1)).dropna()
# compute daily percent change
df_return = df_sp500.pct_change().dropna()
# initize dataframe df_res for tallying end results
df_res = pd.DataFrame(columns=['start', 'end', 'PL VaR', 'PL ES', 'return VaR', 'return ES'])
# keep record of time periods corresponding to calculation
df_res['start'] = df.iloc[:-time_period].index
df_res['end'] = df.iloc[time_period:].index

In [123]:
for i, row in df_res.iterrows():
    # PL in given period
    pl = df_PL.loc[row['start']:row['end']].values
    # PL VaR
    pl_var = np.percentile(pl, q=p)
    df_res.at[i, 'PL VaR'] = pl_var
    # PL ES
    df_res.at[i, 'PL ES'] = np.mean(pl[pl<pl_var])

    # return in given period
    returns = df_return.loc[row['start']:row['end']].values
    # return VaR
    returns_var = np.percentile(returns, q=p)
    df_res.at[i, 'return VaR'] = returns_var
    # return ES
    df_res.at[i, 'return ES'] = np.mean(returns[returns<returns_var])

In [124]:
# note quite by each calendar year but this will be fixed shortly
df_res.iloc[::252]

,start,end,PL VaR,PL ES,return VaR,return ES
0,1995-01-03,1996-01-02,-4.03701,-5.59154,-0.00699934,-0.00991239
252,1996-01-02,1996-12-30,-7.51,-11.49,-0.0110249,-0.0173978
504,1996-12-30,1997-12-29,-14.76,-21.4685,-0.0169246,-0.0241019
756,1997-12-29,1998-12-29,-22.0099,-32.2561,-0.0196425,-0.0302023
1008,1998-12-29,1999-12-29,-23.748,-28.5216,-0.0179887,-0.0217471
1260,1999-12-29,2000-12-27,-30.11,-41.0569,-0.0212179,-0.0290762
1512,2000-12-27,2002-01-03,-25.184,-34.5877,-0.0209463,-0.0293034
1764,2002-01-03,2003-01-03,-24.282,-29.2569,-0.0247886,-0.0313604
2016,2003-01-03,2004-01-05,-14.094,-19.2192,-0.0150828,-0.0210307
2268,2004-01-05,2005-01-04,-14.1699,-16.1108,-0.0121881,-0.0143609
